<a href="https://colab.research.google.com/github/AyaRiad/ayariad/blob/master/Making_a_textbot_Interactive_textgenrnn_Demo_w_GPU_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SECTION 1 
# Scraping text from YouTube videos 


###Install and import API 
The first cell would normally be in your terminal but the ! here allows it to run in the notebook 
The second cell imports it into your code

In [ ]:
!pip install youtube_transcript_api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

Copy and paste the ID's of the videos you want to collect transcripts 

Run the code to save the transcripts to a string called text 

In [ ]:
vid_ids = ['j1rPNNWtbyo', 'PNrlcUMp7rU', 'cGlafnS4uUQ', 'ep5auHgF4-s' , 'C6nutcKVRcU' ,  'RuqXAHU2Ztc', 'xWC6DM465ZQ', '4GYPDB_xVpA', 'acFX7HQKusk', 'lFoq3AbKhs0']
# make sure your video ID's are all in quotations marks and separated by commas

# make an empty string, then iterate through the list of video ID's and each line of text in each trancsript 

text = ''
for i in range (len(vid_ids)): 
  output = YouTubeTranscriptApi.get_transcript(vid_ids[i])
  for j in range(len(output)): 
    text = text + (output[j]['text']+ '\n') 

We can run print to inspect what we've collected 



In [ ]:
print(text)

lovely right okay we're here in front and it's it's a little bit later than we thought because we got to bureau tunnel twenty past three train it's been her parents stop there depending the power to the control center had turned off so that everybody was racked up and waiting and also when you get to front it's another hour behind so we lost an hour anyway so we're we're about two hours where we thought we had two hours per home where we thought we'd be so we've [Music] got flames in fairness which caused some smoke cooling down there yeah so we reassured the tickets repeatedly the bike was not on fire and and he gave a kind of garlic shrug off certain relaxed about it will stay yeah so so well stuff so the drama didn't end there really and as we got going I decided so well I needed to fix this cable and make sure it wasn't us burst into flames as we were leaving and just as I was kind of mucking around with it the cap on the end the tongues that connects to the spark plug just came of

Save to a text file 

In [ ]:
file = open('FILENAME.txt', 'w+')
file.write(text)

52703

# SECTION TWO 
# Generating text 


#  Interactive textgenrnn Demo w/ GPU

by [Max Woolf](http://minimaxir.com)

*Last updated: April 27tht, 2020*

Generate text using a pretrained neural network with a few lines of code, or easily train your own text-generating neural network of any size and complexity, **for free on a GPU using Collaboratory!**

For more about textgenrnn, you can visit [this GitHub repository](https://github.com/minimaxir/textgenrnn).


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip install -q textgenrnn
from google.colab import files
from textgenrnn import textgenrnn
from datetime import datetime
import os

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Set parameters for training 

Please refer to Max's orignal [colab notebook](https://drive.google.com/file/d/1mMKGnVxirJnqDViH7BDJxFqWrsXlPSoK/view?usp=sharing) for his recommended optimal settings. We're using settings here today that will give quick results. 

 (see the [demo jupyter notebook](https://github.com/minimaxir/textgenrnn/blob/master/docs/textgenrnn-demo.ipynb) for more information about these parameters)


In [ ]:
model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 3,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 30,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': True,   # set to True if each text has its own line in the source file
    'num_epochs': 20,   # set higher to train the model for longer 
    'gen_epochs': 10,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any text file** and update the file name in the cell below, then run the cell.

In [ ]:
file_name = "FILENAME.txt"
model_name = 'model_1'   # change to set file name of resulting trained models/texts

The next cell will start the actual training. And thanks to the power of Keras's CuDNN layers, training is super-fast when compared to CPU training on a local machine!

Ideally, you want a training loss less than `1.0` in order for the model to create sensible text consistently.

In [ ]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

We have trained our model! now let's run it to see what happens 

In [ ]:
textgen.generate(n=10, prefix = None, temperature = 0.5)
# the higher the temperature the stranger the text generated

We can run the model in interactive mode and see how the generation process works. It will suggest the top N (default is 3) options for the next character or word.

In [ ]:
textgen.generate(interactive=True, temperature = 0.5, top_n=5)

You can download a large amount of generated text from your model with the cell below! Rerun the cell as many times as you want for even more text!

In [ ]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix = None   # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
  n = 1000
  max_gen_length = 60 if model_cfg['word_level'] else 300
else:
  n = 1
  max_gen_length = 2000 if model_cfg['word_level'] else 10000
  
timestring = datetime.now().strftime('%Y%m%d_%H%M%S')
gen_file = '{}_gentext_{}.txt'.format(model_name, timestring)

textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)


Download and save your text file to your computer 

In [ ]:
files.download(gen_file)

Now you have some text to play with creatively. Use a text to speech program to create a sound file https://www.naturalreaders.com/online/ 

Design an animation etc etc... 

#SECTION 3

In case you want to save any of the models you've trained. You can download the weights below. 

You can download the weights and configuration files in the cell below, allowing you recreate the model on your own computer!

In [ ]:
files.download('{}_weights.hdf5'.format(model_name))
files.download('{}_vocab.json'.format(model_name))
files.download('{}_config.json'.format(model_name))

To recreate the model on your own computer, after installing textgenrnn and TensorFlow, you can create a Python script with:

```
from textgenrnn import textgenrnn
textgen = textgenrnn(weights_path='colaboratory_weights.hdf5',
                       vocab_path='colaboratory_vocab.json',
                       config_path='colaboratory_config.json')
                       
textgen.generate_samples(max_gen_length=1000)
textgen.generate_to_file('textgenrnn_texts.txt', max_gen_length=1000)
```

Have fun with your new model! :)

# Etcetera

If the model fails to load on a local machine due to a model-size-not-matching bug (common in >30MB weights), this is due to a file export bug from Colaboratory. To work around this issue, save the weights to Google Drive with the two cells below and download from there.

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
uploaded = drive.CreateFile({'title': '{}_weights.hdf5'.format(model_name)})
uploaded.SetContentFile('{}_weights.hdf5'.format(model_name))
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1b6T6M32YnXs-c0NB-PEi6MhAdCuG7RHy


If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1